In [2]:
# dl-notes | wad

In [ ]:
# -- micrograd from scratch impl.

In [1]:
# setup
import math
import random

In [15]:
# init. node: holds scalar & gradient
class Node:
    def __init__(self, data, _prev=(), _op=''):
        self.data = data
        self.grad = 0.0
        self._prev = set(_prev)
        self._op = _op
        self._backward = lambda:None

    # addition
    def __add__(self, other):
        other = other if isinstance(other, Node) else Node(other)
        out = Node(self.data + other.data, (self, other), '+')

        def _backward():
            # d p += d child
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward # assign func. sign.

        return out

    # multiplication
    def __mul__(self, other):
        other = other if isinstance(other, Node) else Node(other)
        out = Node(self.data * other.data, (self, other), '*')

        def _backward():
            # d p1 += d child * d p2
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward

        return out

    # power
    def __pow__(self, other):
        assert isinstance(other, (int, float)), "Only supporting int, float powers"
        out = Node(self.data ** other.data, (self,), f'**{other}')

        def _backward():
            # d p += d child * (pw * x**(pw-1))
            self.grad += out.grad * (other * self.data**(other-1))
        out._backward = _backward

        return out

    # activation: ReLU
    def relu(self):
        out = Node(0 if self.data < 0 else self.data, (self,), 'ReLU')

        def _backward():
            # d x = 0(if out = 0), out.grad(if out != 0)
            self.grad += (out.data>0) * out.grad
        out._backward = _backward

        return out

    # main backprop
    def backward(self):
        topo = [] # topology (ordered)
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v) # add parent node after its children
        build_topo(self)

        self.grad = 1 # init. last parent.grad to 1
        for v in reversed(topo): # start from the end
            v._backward()